In [ ]:
import os
import gzip
import torch
import nibabel as nib

# Move working dir to access src
os.chdir("../")

def read_image(file_path):
    nii_img = nib.load(file_path)
    image_data = nii_img.get_fdata()
    header = nii_img.header
    return image_data, header

In [ ]:
image_data, _ = read_image('data_dev/train/images/RibFrac422-image.nii.gz') 
label_data, header = read_image('data_dev/train/labels/RibFrac422-label.nii.gz')

In [ ]:
# Add batch and channel dimensions
image_data = torch.tensor(image_data).unsqueeze(0).unsqueeze(0).float()

In [ ]:
from src.model.models import UNet3D

unet3d = UNet3D(1, 3)

# Run 3D U-Net on [1, 1, 112, 112, 112] volume
data = image_data[:, :, :112, :112, :112]
unet3d_y = unet3d(data)
unet3d_y.shape

In [ ]:
from src.model.models import ResNet183D

resnet183d = ResNet183D(1)

# Run 3D ResNet18 on [1, 1, 56, 56, 56] volume
data = image_data[:, :, :56, :56, :56]
resnet183d_y = resnet183d(data)
[e.shape for e in resnet183d_y]

In [ ]:
from src.model.models import PyramidFeatures3D

fpn3d = PyramidFeatures3D(128, 256, 512)
fpn3d_y = fpn3d(*resnet183d_y)
[e.shape for e in fpn3d_y]

In [ ]:
from src.model.models import RegressionBlock3D

regressgion_block = RegressionBlock3D(256)
regressgion_block_y = torch.cat([regressgion_block(f) for f in fpn3d_y], dim=1)
regressgion_block_y.shape

In [ ]:
from src.model.models import ClassificationBlock3D

classification_block = ClassificationBlock3D(256)
classification_block_y = torch.cat([classification_block(f) for f in fpn3d_y], dim=1)
classification_block_y.shape

In [ ]:
from src.model.anchors import Anchors3D

anchors = Anchors3D()
data = image_data[:, :, :56, :56, :56]
anchors_y = anchors(data)
anchors_y.shape

In [ ]:
from src.model.models import RetinaNet3D

retinanet = RetinaNet3D(1, num_classes=1)
data = image_data[:, :, :56, :56, :56]
retinanet_y = retinanet(data)
[e.shape for e in retinanet_y]

In [ ]:
from src.model.anchors import Anchors3D

anchors = Anchors3D()
data = image_data[:, :, :56, :56, :56]
anchors_y = anchors(data)
anchors_y.shape